<a href="https://colab.research.google.com/github/souhirbenamor/EPF/blob/main/SARIMA_iDiNa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [ ]:
data = pd.read_excel('/content/normalized_df_32.xlsx')
data.drop(data.loc[data['Date'].duplicated()].index, inplace = True)
#df['datetime'] = pd.to_datetime(df['Date'])

# df = df.sort_values(df['datetime'], ascending=[True])

data.set_index(data['Date'], inplace = True)
data.drop(columns = ['Date'], inplace = True)
data.tail()

,N1,N2,N3,N4,N5,N6,N7,N8,N9,N10,...,N23,N24,N25,N26,N27,N28,N29,N30,N31,N32
Date,,,,,,,,,,,,,,,,,,,,,
2021-12-31 20:00:00,0.060440,0.061210,0.159555,0.029960,0.031795,0.083620,0.114880,0.037960,0.029280,0.025000,...,0.277850,0.276330,0.023435,0.035390,0.037175,0.134915,0.145420,0.065955,0.135425,0.026525
2021-12-31 21:00:00,0.056300,0.061495,0.160530,0.022200,0.030705,0.090710,0.084410,0.036635,0.025665,0.022065,...,0.237025,0.245445,0.025760,0.025705,0.022610,0.095620,0.101230,0.052910,0.116535,0.031065
2021-12-31 22:00:00,0.058155,0.023990,0.115475,0.024035,0.024600,0.083975,0.088615,0.026980,0.022535,0.017320,...,0.219075,0.211725,0.023135,0.020335,0.023095,0.085310,0.091030,0.040985,0.101180,0.020270
2021-12-31 23:00:00,0.059620,0.069505,0.084865,0.021390,0.020855,0.085335,0.090890,0.023725,0.019830,0.061090,...,0.224850,0.206835,0.020670,0.019025,0.022980,0.070480,0.088010,0.086130,0.118370,0.017315
2022-01-01 00:00:00,0.046720,0.072180,0.039605,0.019195,0.019340,0.063415,0.084100,0.024790,0.021390,0.044820,...,0.213000,0.161015,0.020040,0.015875,0.020900,0.059450,0.069245,0.081545,0.090965,0.016410


In [ ]:
# Load the data
df = data[["N1"]]

In [ ]:


# Define the training and testing sample sizes
train_size = 26300

# Split the data into train and test sets
train = df.iloc[:train_size]
test = df.iloc[train_size:]

# Define the rolling window length
window_length = 24

# Create empty lists to store the actual and predicted values
actual_values = []
predicted_values = []

# Loop over each rolling window and predict the next value in the time series
for i in range(window_length, len(test)):
    
    # Get the current test set window
    current_test = test.iloc[i-window_length:i]
    
    # Fit the SARIMA model to the train set
    model = SARIMAX(train, order=(1, 0, 1), seasonal_order=(1, 0, 1, 24))
    results = model.fit()
    
    # Predict the next value in the time series using the fitted model
    y_pred = results.forecast(steps=1)
    



/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/stat

In [ ]:
# Append the actual and predicted values to their respective lists
actual_values.append(test.iloc[i,0])
predicted_values.append(y_pred[0])   
# Append the predicted value to the train set for the next iteration
train = train.append(pd.DataFrame(y_pred, columns=["N1"]))

In [ ]:
# Calculate the evaluation metrics
mse = mean_squared_error(actual_values, predicted_values)
mae = mean_absolute_error(actual_values, predicted_values)
r2 = r2_score(actual_values, predicted_values)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-Squared (R2):", r2)

# Plot the actual and predicted values
plt.figure(figsize=(12,6))
plt.plot(df, label="Actual")
plt.plot(test.index[window_length:], predicted_values, label="Predicted")
plt.legend()
plt.show()